# Starting to explore daily train data

In [3]:
import pandas as pd
import numpy as np
import os
import sys
import datetime

# Hack for using import with relative path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from fetchTrainData import fetch_train_data
from fetchWeatherData import fetch_weather_data

date = datetime.date.today() - datetime.timedelta(days=0)

# Fetch trains for a given station
q = (
    """ 
    {
    """
      f'  trainsByDepartureDate(departureDate: \"{date}\",'
    """
        take: 500, 
        where: {
            and: [
                {or: [{deleted: {unequals: null}}, 
                {deleted: {equals: false}}]}, 
                {cancelled: {equals: false}}, 
                {operator: {shortCode: {equals: "vr"}}}
                ]  
        }
      ) 
      {
        trainNumber
        departureDate
        commuterLineid
        trainType {
          name
          trainCategory {
            name
          }
        }
        timeTableRows {
        differenceInMinutes
          station {
            shortCode
            location
          }
        }
      }
    }
    """
    )

result = fetch_train_data(q)

recs = result['data']['trainsByDepartureDate']
recs

[{'trainNumber': 1,
  'departureDate': '2021-10-23',
  'commuterLineid': '',
  'trainType': {'name': 'IC', 'trainCategory': {'name': 'Long-distance'}},
  'timeTableRows': [{'differenceInMinutes': None,
    'station': {'shortCode': 'HKI', 'location': [24.94125, 60.172096]}},
   {'differenceInMinutes': None,
    'station': {'shortCode': 'PSL', 'location': [24.933521, 60.19869]}},
   {'differenceInMinutes': None,
    'station': {'shortCode': 'PSL', 'location': [24.933521, 60.19869]}},
   {'differenceInMinutes': None,
    'station': {'shortCode': 'KÄP', 'location': [24.946033, 60.22017]}},
   {'differenceInMinutes': None,
    'station': {'shortCode': 'KÄP', 'location': [24.946033, 60.22017]}},
   {'differenceInMinutes': None,
    'station': {'shortCode': 'OLK', 'location': [24.967676, 60.229]}},
   {'differenceInMinutes': None,
    'station': {'shortCode': 'OLK', 'location': [24.967676, 60.229]}},
   {'differenceInMinutes': None,
    'station': {'shortCode': 'PMK', 'location': [24.993288, 

In [7]:
df = pd.json_normalize(recs)
all_stops = pd.json_normalize(df['timeTableRows'])
df = df.drop(['timeTableRows'], axis=1)

weather_data = {}

def get_weather(tag, lat=0, lon=0):
    if tag in weather_data:
        print("DATA FOUND")
        return weather_data[tag]
    
    print("FETCHING DATA..")
    weather_data[tag] = fetch_weather_data(lat, lon, date)

def extract_time_tables(train_stops_):
    train_stops = pd.json_normalize(train_stops_)
    to_return = train_stops['differenceInMinutes'].sum()
    train_stops = train_stops.drop(['differenceInMinutes'], axis=1)

    train_stops.apply(lambda x : get_weather(x['station.shortCode'], x['station.location'][0], x['station.location'][1]), axis=1)
        
    for stop in train_stops['station.shortCode'].unique():
        if stop not in df.columns:
            df[stop] = None
        
        df[stop][train_stops_.name] = 1
    
    return to_return

how_much_late = all_stops.apply(extract_time_tables, axis=1)
df

FETCHING DATA..


Exception: Query failed to run with a 401